In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_orig = pd.read_csv('./credit_train.csv', sep=';', index_col='client_id')

df = df_orig.copy(deep=True)
target = df['open_account_flg']
# pd.Series(df_orig["living_region"].unique()).to_csv('./regions.csv')

In [8]:
from sklearn_pandas import CategoricalImputer
import sanitizer

df = sanitizer.sanitize_frame(df)

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(df.corr())
plt.show()
df.corr()
# Видим, что наибольшая корреляция у признаков tariff_id и score_shk - 0.398

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

encoder = preprocessing.LabelEncoder()

for dtype in df.dtypes.index:
    if df.dtypes[dtype] == 'object':
        df[dtype] = encoder.fit_transform(df[dtype])  
df.columns

In [ ]:
def one_hot_encode(frame, column_name):
    series = frame[column_name]
    series_dummies = pd.get_dummies(series, prefix=column_name)
    new_frame = frame.join(series_dummies)
    new_frame = new_frame.drop(columns=[column_name])
    
    return new_frame
    
df = one_hot_encode(df, 'gender')
df = one_hot_encode(df, 'credit_month')
df = one_hot_encode(df, 'marital_status')
df = one_hot_encode(df, 'education')
df = one_hot_encode(df, 'job_position')
df = one_hot_encode(df, 'living_region')

df.columns

In [ ]:

# Нормализуем credit_sum, monthly_income

def normalize(series):
    return (series - series.mean()) / series.std()
    
df["credit_sum"] = normalize(df["credit_sum"])
df["monthly_income"] = normalize(df["monthly_income"])
df["tariff_id"] = normalize(df["tariff_id"])
df["score_shk"] = normalize(df["score_shk"])

df.columns

In [ ]:
dfe = df[['tariff_id', 'score_shk', 'open_account_flg']].copy()[:500]

X = dfe.iloc[:,:-1]
y = dfe.iloc[:,-1]

admitted = dfe.loc[y==1]
not_admitted = dfe.loc[y==0]

plt.scatter(admitted.iloc[:, 0], admitted.iloc[:, 1], s=10, label='Admitted')
plt.scatter(not_admitted.iloc[:, 0], not_admitted.iloc[:, 1], s=10, label='Not Admitted')
plt.legend()
plt.show()



In [ ]:
# df = df.drop(columns=["marital_status", "job_position", "score_shk", "education", "living_region", "overdue_credit_count"])
# df.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df.drop(columns=['open_account_flg'])
y = df['open_account_flg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=34)
clf = LogisticRegression(random_state=2, solver='lbfgs', max_iter=2500)

clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_test_pred = clf.predict(X_test)
# 
# print("Accuracy - {}".format(clf.score(X_test, y_test)))
print(confusion_matrix(y_test, y_test_pred))
# print(classification_report(y_test, y_test_pred))
print(f1_score(y_test, y_test_pred))
clf.coef_
# Accuracy почти не изменилась. Confusion matrix показывает, что модель стала определять true negative и это уже хорошо

df.head()